## Radios en Circuitos

**En esta notebook se usan los archivos:**

Radios censales 2010 por IGN

- `radios = './radios_IGN_2010_WGS84/'`

Circuitos compilados por Tartagalensis en base a CNE

- `circuitos_ARG = './mapaelectoral/circuitos-CNE-TTGL.geojson'`

Geometrías de Dptos y Provincias de IGN

- `dptos_url = 'https://dnsg.ign.gob.ar/apps/api/v1/capas-sig/Geodesia+y+demarcación/Límites/departamento/json'`

- `provs_url = 'https://dnsg.ign.gob.ar/apps/api/v1/capas-sig/Geodesia+y+demarcación/Límites/provincia/json'`


**Y se generan los archivos:**

Secciones según circuitos de Tartagalensis - CNE

```python
prov_dpto_IN1[['codprov', 'coddepto', 'IN1']].to_csv('./info/secciones_departamentos-TTGL.csv', index=False)

ref_dptos[['in1prov', 'namprov', 'in1', 'nam', 'IN1', 'geometry']].to_file('./mapaelectoral/ref-dptosIGN.geojson', driver='GeoJSON')

ref_dptos[['in1prov', 'namprov', 'in1', 'nam', 'IN1']].to_csv('./info/ref-dptosIGN.csv', index=False)


In [76]:
import pandas as pd
import geopandas as gpd

import warnings
warnings.filterwarnings('ignore')

from shapely.validation import make_valid

def fix_geom(in_feature):

    # avoid changing original geodf
    in_feature = in_feature.copy(deep=True)    
        
    # drop any missing geometries
    in_feature = in_feature[~(in_feature.is_empty)]
    
    # Repair broken geometries
    for index, row in in_feature.iterrows(): # Looping over all polygons
        if row['geometry'].is_valid:
            next
        else:
            fix = make_valid(row['geometry'])

            try:
                in_feature.loc[[index],'geometry'] =  fix # issue with Poly > Multipolygon
            except ValueError:
                in_feature.loc[[index],'geometry'] =  in_feature.loc[[index], 'geometry'].buffer(0)
    return in_feature

## Cargar geometrias

In [77]:
radios = gpd.read_file('./radios_IGN_2010_WGS84/')

In [78]:
# circuitos_ARG = gpd.read_file('./mapaelectoral/circuitos-CNE.geojson')
circuitos_ARG = gpd.read_file('./mapaelectoral/circuitos-CNE-TTGL.geojson')

# Radios en Circuitos
#### Interseccion espacial

In [79]:
circuitos_ARG.count()

circuito    5589
codprov     5563
coddepto    5563
seccion     1066
geometry    5559
dtype: int64

In [80]:
circuitos_ARG.nunique()

circuito    2058
codprov       24
coddepto     138
seccion        8
geometry    5550
dtype: int64

In [81]:
## Agregar codigos de provincia de INDEC
codprovs = pd.read_csv('./codprovs.csv', dtype = 'str')
codprovs['codprov'] = codprovs['codprov'].str.zfill(2)

circuitos_ARG = circuitos_ARG.merge(codprovs, how = 'left')

In [82]:

result_list = []

for indec_p in circuitos_ARG.indec_p.dropna().unique():
    cne_prov = circuitos_ARG.loc[circuitos_ARG.indec_p.fillna(indec_p) == indec_p]
    cne_prov = fix_geom(cne_prov.dropna(subset = 'geometry'))
    print(cne_prov.nomprov.unique())

    radios_prov = radios.loc[radios.PROV_ == indec_p]

#     ## Correccion de errores
#     radios_prov.loc[radios_prov.IN1 == '26028', 'DEPTO_'] = '063' # Esto es para adaptarse a un erorr en CNE
# #             (la CNE tiene unido camarones al dpto martires, por error)

    result_prov = []

#     for dpto in cne_prov.indec_d.unique():

#         if (cne_prov.indec_p.unique()[0] == '94') | (cne_prov.indec_p.unique()[0] == '22'):
# #                     print('chaco o tdf')
#             radios_dpto = radios_prov # En chaco y tdf, usar toda la provincia porque los codigos estan cambiados...
#         else:
#             radios_dpto = radios_prov.loc[radios_prov.DEPTO_ == dpto]

# #         circ_dpto = cne_prov.loc[cne_prov.indec_d == dpto]

# #     Check...
#     fig, axs = plt.subplots(1)
#     radios_prov.plot(ax = axs, alpha = .4)
#     cne_prov.plot(ax = axs, alpha = .4)
#     plt.show()

    ## Intersectar los radios en cuestion con los polis de IGS.
    ref_dptos = gpd.overlay(radios_prov, cne_prov[['circuito', 'codprov', 'coddepto', 'geometry']], how='intersection')
    
    
    if len(ref_dptos) != 0: # Si hay interseccion
        ref_dptos['area'] = ref_dptos.area

        year = '2010'
        # Quedarse con el que mas interseca (el radio se asigna al depto donde este mayor parte de su area.)
        radios_in_circu = ref_dptos.groupby(['PROV_', 'DEPTO_', 'FRACC_', 'COD_'+year]
                                        ).apply(lambda x: x.nlargest(1, 'area')).reset_index(drop = True
                    )[['codprov', 'coddepto', 'PROV_', 'DEPTO_', 'FRACC_', 'COD_'+year, 'IN1', 'circuito']]
        
        # radios_in_circu['FRACC_2'] = radios_in_circu['COD_'+year].str[5:7]
        result_prov += [radios_in_circu]
    else:
#             print('sin interseccion en '+str(circ_dpto.departamen.unique()[0]))
        print('sin interseccion en '+str(cne_prov.nomprov.unique()[0]))

    result_prov = pd.concat(result_prov)

    ## Tomar radios que se asociaron a mas de 1 circuito.
#     Esto pasa cuando se usa la provincia entera para intersecar (ie. chaco, tdf)
    duplicados = result_prov[result_prov.duplicated('COD_2010', keep = False)].sort_values('COD_2010')

    if len(duplicados) > 0:

        ## Volcarlos en 1 solo circuito.
        ref_dptos = gpd.overlay(gpd.GeoDataFrame(duplicados.drop('circuito', axis = 1).merge(radios)), 
                             gpd.GeoDataFrame(duplicados.merge(cne_prov))[['circuito', 'geometry']], how='intersection')

        if len(ref_dptos) != 0: # Si hay interseccion
            ref_dptos['area'] = ref_dptos.area

            year = '2010'
            # Quedarse con el que mas interseca (el radio se asigna al depto donde este mayor parte de su area.)
            res_dup_prov = ref_dptos.groupby(['PROV_', 'DEPTO_', 'FRACC_', 'COD_'+year]
                                        ).apply(lambda x: x.nlargest(1, 'area')
                                               ).reset_index(drop = True)[['PROV_', 'DEPTO_', 'FRACC_', 'COD_'+year, 'IN1', 'circuito']]
        result_prov = pd.concat([result_prov[~result_prov.duplicated('COD_2010', keep = False)], res_dup_prov])

    result_list += [result_prov]

result = pd.concat(result_list).drop_duplicates()


['Ciudad Autónoma de Buenos Aires' nan]
['Catamarca' nan]
['Chaco' nan]
['Chubut' nan]
['Córdoba' nan]
['Corrientes' nan]
[nan 'Entre Ríos']
[nan 'Formosa']
[nan 'Jujuy']
[nan 'La Pampa']
[nan 'La Rioja']
[nan 'Mendoza']
[nan 'Misiones']
[nan 'Neuquén']
[nan 'Buenos Aires']
[nan 'Río Negro']
[nan 'Salta']
[nan 'San Luis']
[nan 'Santa Cruz']
[nan 'Santa Fe']
[nan 'Santiago del Estero']
[nan 'San Juan']
[nan 'Tierra del Fuego, Antártida e Islas del Atlántico Sur']
[nan 'Tucumán']


In [83]:
radios.nunique()

PROV_          24
DEPTO_        154
FRACC_         99
COD_2010    52401
IN1           525
geometry    52401
dtype: int64

In [84]:
result.nunique()

codprov        24
coddepto      138
PROV_          24
DEPTO_        154
FRACC_         99
COD_2010    52159
IN1           525
circuito     1974
dtype: int64

In [85]:
# Esto es lo extraido de la base
# seccion_dpto = pd.read_csv('./../elecciones-ARG/datos/BD/seccion_table.csv', dtype = 'str')
# seccion_dpto['distrito_id'] = seccion_dpto['distrito_id'].str.zfill(2)
# seccion_dpto['seccion_id'] = seccion_dpto['seccion_id'].str.zfill(3)
# seccion_dpto.nunique()

## Este archivo esta basado en lo extraido de la base, pero trae los codigos usados en INDEC, etc (PROV_, DEPTO_)
# result = pd.read_csv('./info/radios_circuitos-TTGL.csv', dtype= 'str')
secciones_dptos_ref = pd.read_csv('./info/secciones_dptos_ref.csv')

In [86]:
secciones_dptos_ref = secciones_dptos_ref.dropna()
secciones_dptos_ref['PROV_'] = secciones_dptos_ref.IDPROV.astype(int).astype(str).str.zfill(2)
secciones_dptos_ref['DEPTO_'] = secciones_dptos_ref.IDDPTO.astype(int).astype(str).str.zfill(3)

In [87]:
claves = secciones_dptos_ref[['PROV_', 'DEPTO_', 'distrito_id', 'seccion_id', 'seccionprovincial_id', 'seccion_nombre']]
claves.head()

,PROV_,DEPTO_,distrito_id,seccion_id,seccionprovincial_id,seccion_nombre
0,02,001,1,1,0,Comuna 01
1,02,002,1,2,0,Comuna 02
2,02,003,1,3,0,Comuna 03
3,02,004,1,4,0,Comuna 04
4,02,005,1,5,0,Comuna 05


In [88]:
result.head()

,codprov,coddepto,PROV_,DEPTO_,FRACC_,COD_2010,IN1,circuito
0,01,001,02,001,01,020010101,0214007,00005
1,01,001,02,001,02,020010201,0214007,00005
2,01,001,02,001,02,020010202,0214007,00005
3,01,001,02,001,02,020010203,0214007,00005
4,01,001,02,001,02,020010204,0214007,00005


In [89]:
result_c = result.merge(claves)

In [90]:
result_c.to_csv('./info/radios_circuitos_secciones_ref.csv', index = False)

In [91]:
result_c.head()

,codprov,coddepto,PROV_,DEPTO_,FRACC_,COD_2010,IN1,circuito,distrito_id,seccion_id,seccionprovincial_id,seccion_nombre
0,01,001,02,001,01,020010101,0214007,00005,1,1,0,Comuna 01
1,01,001,02,001,02,020010201,0214007,00005,1,1,0,Comuna 01
2,01,001,02,001,02,020010202,0214007,00005,1,1,0,Comuna 01
3,01,001,02,001,02,020010203,0214007,00005,1,1,0,Comuna 01
4,01,001,02,001,02,020010204,0214007,00005,1,1,0,Comuna 01


## Codprov - Coddepto - IN1

In [92]:
pd.options.display.max_rows = 99

In [93]:
## Cuando codprov y coddpto tienen mas de un IN1, elegir el que mas radios tiene en la interseccion.

prov_dpto_nuniques = result.groupby(['codprov', 'coddepto', 'IN1']).nunique().reset_index()
prov_dpto_nuniques['pct'] = prov_dpto_nuniques.groupby(['codprov', 'coddepto'])['COD_2010'].apply(lambda x: x/x.sum()).round(2)
prov_dpto_nuniques['test'] = abs(prov_dpto_nuniques['pct'] - .5)

# test.sort_values('test').head(20).sort_values('IN1')

TypeError: incompatible index of inserted column with frame index

In [ ]:
## Lezama y Chascomus
prov_dpto_nuniques.loc[prov_dpto_nuniques.IN1 == '06217']

: 

In [ ]:
## Tierra del Fuego
prov_dpto_nuniques.loc[prov_dpto_nuniques.IN1.str[:2] == '94']

: 

In [ ]:
prov_dpto_IN1 = prov_dpto_nuniques.groupby(['codprov', 'coddepto']).apply(lambda x: x.nlargest(1, 'COD_2010')).reset_index(drop = True)

prov_dpto_IN1[['codprov', 'coddepto', 'IN1']].to_csv('./info/secciones_departamentos-TTGL.csv', index = False)

: 

In [ ]:
prov_dpto_IN1[['codprov', 'coddepto', 'IN1']].head()

: 

## Unir secciones a departamentos IGN

In [ ]:
import geopandas as gpd

# Unir secciones a departamentos IGN
radios_diss_IN1 = radios.copy()
radios_diss_IN1['geometry'] = radios_diss_IN1.buffer(0.001)
radios_diss_IN1 = radios_diss_IN1.dissolve('IN1').reset_index()[['IN1', 'geometry']]

# Si bien en teoria este dissolve nos daria los departmantos, el archivo de radios IGN tiene varios faltantes y errores.
# Por eso es mejor hacer spatial join de los radios disueltos, con la fuente de DPTOS.

: 

### Radios de IGN en DPTOS y PROVS de IGN

In [ ]:
## Si este paso falla, se pueden bajar manualmente los archivos y abrirlos localmente.

try: 
    dptos_url = u'https://dnsg.ign.gob.ar/apps/api/v1/capas-sig/Geodesia+y+demarcación/Límites/departamento/json'
    dptos_IGN = gpd.read_file(dptos_url)

    # Agregar código y nombres de provincia
    provs_url = 'https://dnsg.ign.gob.ar/apps/api/v1/capas-sig/Geodesia+y+demarcación/Límites/provincia/json'
    provs_IGN = gpd.read_file(provs_url)

except:
    dptos_IGN = gpd.read_file('./datos/departamento.json')
    provs_IGN = gpd.read_file('./datos/provincia.json')


: 

In [ ]:

dptos_IN1_from_radios = radios_diss_IN1

ref_dptos = gpd.overlay(dptos_IGN[['nam', 'in1', 'geometry']], dptos_IN1_from_radios, how='intersection')
ref_dptos['area'] = ref_dptos.area

ref_dptos = ref_dptos.groupby(['nam', 'in1']).apply(lambda x: x.nlargest(1, 'area')).reset_index(drop=True)

provnames = provs_IGN.rename(columns={'in1': 'in1prov', 'nam': 'namprov'})[['in1prov', 'namprov']]
ref_dptos['in1prov'] = ref_dptos['in1'].str[:2]
ref_dptos = ref_dptos.merge(provnames, on='in1prov')

ref_dptos.head()

: 

In [ ]:
# Lugares con código diferente (in1, de radios IGN, vs. IN1 de departamentos IGN)
# Son las comunas de CABA, además de Lezama-Chascomús y Tierra del Fuego
ref_dptos.loc[ref_dptos.in1 != ref_dptos.IN1].sort_values('in1')

: 

### Guardar los resultados

In [ ]:
# Geojson
ref_dptos[['in1prov', 'namprov', 'in1', 'nam', 'IN1', 'geometry']].to_file('./mapaelectoral/ref-dptosIGN.geojson', driver='GeoJSON')

# CSV
ref_dptos[['in1prov', 'namprov', 'in1', 'nam', 'IN1']].to_csv('./info/ref-dptosIGN.csv', index=False)

: 

: 